In [206]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

import imageio as iio
from scipy.interpolate import interp1d
from sys import maxsize
from itertools import permutations, product

from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits

import pandas as pd
import pickle

from tqdm import tqdm

import imageio
import os

In [311]:
def calc_distance_slew_time(f1,f2):
    d = np.sqrt((f1[0]-f2[0])**2. + (f1[1]-f2[1])**2.) 
    return get_slew_time(d)


def travel_salesman(s, fields):
    V=len(fields)
    vertex = []
    for i in range(V):
        if i != s:
            vertex.append(i)
    min_cost = maxsize
    next_permutation=permutations(vertex)
    for i in next_permutation:
        current_cost = 0
        k = s
        for j in i:
            current_cost += calc_distance_slew_time(fields[k],fields[j])
            k = j
        current_cost += calc_distance_slew_time(fields[k],fields[s])
        min_cost = min(min_cost, current_cost)
        
    return min_cost



def get_minimum_cadence(fields, nreads=19., readtime=3.04):
    
    slewing_time = travel_salesman(0, fields)    
    exposure_time = len(fields)*nreads*readtime
    
    return (slewing_time + exposure_time)/60.




def plot_roman_footprint(field_center, ax=None, **kw_arg):
    dl, db = np.loadtxt('roman_field_outlines.txt', skiprows=1, delimiter=',').T
    if not(ax is None):
        ax.plot(field_center[0]+dl, field_center[1]+db, **kw_arg)
    else:
        plt.plot(field_center[0]+dl, field_center[1]+db, **kw_arg)




def get_pointing_info(fields, n_frames=150, exptime=57.,  settle_time=11.1):
    
        
    time_limits = np.zeros(len(fields)+1)
    t0=0.
    
    for i,(l_0, b_0) in enumerate(fields):
        
        l_1, b_1 = fields[(i+1)%(len(fields)-1)]


        slew_time = calc_distance_slew_time((l_0, b_0), (l_1,b_1) ) - settle_time
        t0 += slew_time + settle_time + exptime
        time_limits[i+1] += t0
                
    
    times = np.linspace(0,time_limits[-1], n_frames)
    
    
    ls=[]
    bs=[]
    modes=[]
    times=[]
    
    escape_counter=0
    field_num=0
    dt = time_limits[-1]/n_frames
    t=0
        
    while field_num < len(fields) and escape_counter<1e5:
                
        if t<time_limits[field_num+1]:
            
            if t-time_limits[field_num] < settle_time:
                modes.append('settling...')
                
                ls.append(fields[field_num][0])
                bs.append(fields[field_num][1])
                
            elif t-time_limits[field_num] < settle_time+exptime:
                modes.append('exposing...')
                ls.append(fields[field_num][0])
                bs.append(fields[field_num][1])
                
            else:
                modes.append('slewing...')
                
                
                l1 , b1 = fields[field_num]
                l2 , b2 = fields[(field_num+1)%(len(fields))]
                                                
                slew_time = time_limits[field_num+1]-settle_time-exptime-time_limits[field_num]
                slew_progress = t-settle_time-exptime-time_limits[field_num]
                                
                ls.append(l1 + (l2-l1) * (slew_progress/slew_time)  )
                bs.append(b1 + (b2-b1) * (slew_progress/slew_time)  )

            
            t+=dt
            times.append(t)
            
        else:
            field_num+=1
    
            
    
    
    return times, ls, bs, modes






def make_images_and_gif(pointing_info, field_centers, reddening_map, save_direc='nominal_6fields_plus_GC', 
                        slew_color='tomato', settle_color='orange', exposure_color='dodgerblue'):

    t_animated, l_animated, b_animated, obsmodes = pointing_info


    for i in tqdm(range(len(t_animated)) ): 

        ###################################################
        # PLOT THE BASE IMAGE HERE
        plt.figure(figsize=(6.,4.5))

        A_Ks = reddening_map['e_jk'] * 0.428
        A_H = A_Ks * 1.88


        cmap = mpl.cm.Greys
        #bounds = np.arange(0., 5.25, 0.05)
        #norm = mpl.colors.BoundaryNorm(bounds, cmap.N, )
        
        plt.pcolormesh(reddening_map['l'], reddening_map['b'], A_H, cmap=cmap, vmin=0, vmax=5)#norm=norm)
        plt.colorbar(label='$\mathregular{A_H}$ [mag]', )
        
        plt.plot(0,0,'*', markeredgecolor='k', markersize=15, color='w')
        
        for f in field_centers:
            plot_roman_footprint(f, ls='-', color='k', lw=1)
        
        
        plt.xticks([-3,-2,-1,0,1,2,3])
        plt.xlim(-3,3)
        
        plt.ylim(-3.,3.)
        plt.xlim(3.,-3.)
        
        plt.xlabel('$\mathregular{l \;[\circ]}$')
        plt.ylabel('$\mathregular{b \;[\circ]}$')  
        #################################################
        
        
        l_pointing = l_animated[i]
        b_pointing = b_animated[i]
        
        
        if obsmodes[i]=='slewing...':
            obscolor=slew_color
        
        elif obsmodes[i]=='settling...':
            obscolor=settle_color
        
        else:
            obscolor=exposure_color
        
        plot_roman_footprint( (l_animated[i], b_animated[i]), color=obscolor, ls='-', lw=3)
        
        if i%5!=0:
            plt.text(0.05,0.95, obsmodes[i], color=obscolor, fontsize=15, ha='left', va='top', 
                 transform=plt.gca().transAxes)
        plt.text(0.95,0.95, '{:.1f} min'.format(t_animated[i]/60. ), color=obscolor, fontsize=15, 
                 ha='right', va='top', transform=plt.gca().transAxes)
        
        plt.tight_layout()
        plt.savefig(save_direc+'/roman_gbtds_animation'+'{}'.format(i).zfill(4)+'.jpg',dpi=100, )
        plt.close()



    images = []
    for file_name in sorted(os.listdir(save_direc)):
        if file_name.endswith('.jpg'):
            file_path = os.path.join(save_direc, file_name)
            images.append(imageio.imread(file_path))


    imageio.mimsave('./'+save_direc+'.gif', images, loop=0, duration=100)



In [312]:
# Load all Relevant Data
slew_settle_data = np.loadtxt('slew_settle_times_NEW.txt').T
get_slew_time = interp1d(x=slew_settle_data[0], y=slew_settle_data[1], kind='cubic')

field_centers = pd.read_csv('field_centers_nominal_6plusGC.txt', ).to_numpy()

f = open('surot_2020.pkl', 'rb') 
reddening_map = pickle.load(f)


In [313]:
observing_info = get_pointing_info(field_centers, n_frames=200, exptime=57.,  settle_time=11.1 )
make_images_and_gif(observing_info, field_centers, reddening_map)

100%|█████████████████████████████████████████| 200/200 [00:41<00:00,  4.82it/s]
/var/folders/5f/j23qdt652zv1ykr4dgp6jt1rjmmlsv/T/ipykernel_82815/1249418412.py:203: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))
